In [2]:
import numpy as np
import os
from scipy.io import wavfile
from python_speech_features import mfcc, logfbank
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from keras.layers import LSTM, Dense, TimeDistributed
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
from keras.layers import Input, Dense
from keras.models import Model
%matplotlib inline
import json
import pandas as pd
from keras.models import Sequential

from utils import *

pd.options.display.max_columns = None
pd.set_option('compute.use_bottleneck', True)
pd.set_option('compute.use_numexpr', True)

In [3]:
data_header='target'
for i in range(2808): data_header += ',' + str(i)

#### Get data

In [4]:
# dataset
dataset='ADA'

fpaths = []
labels = []
boy_fpaths = []
boy_labels = []

word_spoken = []

dataset = '41{}'.format(dataset)
input_folder = '..\data\{}'.format(dataset)

# pars the input directory that contains audio files
# get audio files and their lables

for f in os.listdir(input_folder):
    for w in os.listdir(input_folder+'\\'+ f):
        # check wheter files is wav or not

        if (w.find('wav')!=-1):
            fpaths.append(input_folder+'\\'+f+'\\'+w)
            labels.append(f)
        
#             idx_schunk=w.find('schunk')
#             idx_chunk=w.find('chunk')
            
#             if (idx_chunk!=idx_schunk==-1):
#                 if (int(w[idx_chunk-2:idx_chunk])<80):
#                     fpaths.append(input_folder+'\\'+f+'\\'+w)
#                     labels.append(f)
#                 else:
#                     boy_fpaths.append(input_folder+'\\'+f+'\\'+w)
#                     boy_labels.append(f)
        
#             else:
#                 boy_fpaths.append(input_folder+'\\'+f+'\\'+w)
#                 boy_labels.append(f)

                
            if f not in word_spoken:
                word_spoken.append(f)
print("Spoken words: "+ str(word_spoken))

Spoken words: ['0_zero', '1_one', '2_two', '3_three', '4_four', '5_five', '6_six', '7_seven', '8_eight', '9_nine', 'aboard', 'adjusted & locked', 'All switches', 'Alternate air door', 'A_C Documents', 'Battery+Main bus', 'Cabin doors', 'Checked', 'Circuit breakers', 'Closed', 'Cockpit', 'Cockpit cheklist completed', 'Completed', 'Decimal', 'Flight Controls', 'Fuel Quantity', 'Fuel Selector', 'Fuel Shutoff Valve', 'Fuel Temperature', 'in', 'locked', 'off', 'On', 'open', 'preflight inspection', 'removed', 'Seats & Belts', 'Shut-off cabin heat', 'sufficient', 'Towbar', 'Weight and balance']


In [5]:
# size of dataset
print(len(labels))

9187


In [6]:
# print(len(boy_labels))

In [7]:
label_set=list(set(labels))
label_set.sort()

# # boys 
# boy_label_set=list(set(boy_labels))
# boy_label_set.sort()

In [8]:
label_set

['0_zero',
 '1_one',
 '2_two',
 '3_three',
 '4_four',
 '5_five',
 '6_six',
 '7_seven',
 '8_eight',
 '9_nine',
 'A_C Documents',
 'All switches',
 'Alternate air door',
 'Battery+Main bus',
 'Cabin doors',
 'Checked',
 'Circuit breakers',
 'Closed',
 'Cockpit',
 'Cockpit cheklist completed',
 'Completed',
 'Decimal',
 'Flight Controls',
 'Fuel Quantity',
 'Fuel Selector',
 'Fuel Shutoff Valve',
 'Fuel Temperature',
 'On',
 'Seats & Belts',
 'Shut-off cabin heat',
 'Towbar',
 'Weight and balance',
 'aboard',
 'adjusted & locked',
 'in',
 'locked',
 'off',
 'open',
 'preflight inspection',
 'removed',
 'sufficient']

## Extracting frequeny domain features

At the second stage we convet a signal into the freqency domain. In monst modern speech recognitoon freqeency-domain features are used as key component. In case of multispeakers MFFC feature extraction works best. After convert a signal into a freq domain, it's requered to convert it into a useable form. **Mel Frequency Cepstral Coefficients (MFCC)** is a good way to do that. *MFCC* takes the power spectrum of a signal and then uses a combination of filter banks and disrete cosinetransform to extract pattern of phones or features.

After extracting **MFFC** features we exract data into single data matrix, and a label vector with the correct label for eac data file is ceated. 

In [9]:
from scipy.io import wavfile

def get_mfcc(fpaths):
    data = []

    mfcc_max_length = 0

    # first file desitination name and index
    file_name = ''
    word_spoken_index = 0

    for n,file in enumerate(fpaths):

        # show current desintation 
        if (file.find(file_name)<=0):
            file_name=word_spoken[word_spoken_index]
            print(word_spoken[word_spoken_index])
            word_spoken_index+=1

        # read file 
        sampling_freq, audio = wavfile.read(file)
        # Extract MFCC features
        mfcc_features = mfcc(audio, sampling_freq)

        mfcc_len=mfcc_features.shape[0]*mfcc_features.shape[1]
        # get length of largets feature array
        if mfcc_len>mfcc_max_length:    
            mfcc_max_length=mfcc_len


        # flat data into 2D array
        mfcc_features=np.resize(mfcc_features,(1,mfcc_len))

        data.insert(n,mfcc_features)
        
    return data

In [10]:
data=get_mfcc(fpaths)
# boy_data=get_mfcc(boy_fpaths)

0_zero
1_one
2_two
3_three
4_four
5_five
6_six
7_seven
8_eight
9_nine
aboard
adjusted & locked
All switches
Alternate air door
A_C Documents
Battery+Main bus
Cabin doors
Checked
Circuit breakers
Closed
Cockpit
Cockpit cheklist completed
Completed
Decimal
Flight Controls
Fuel Quantity
Fuel Selector
Fuel Shutoff Valve
Fuel Temperature
in
locked
off
On
open
preflight inspection
removed
Seats & Belts
Shut-off cabin heat
sufficient
Towbar
Weight and balance


#### Get Labels

In [11]:
def get_labels(labels, label_set):
    label_idx_dic={} # dict for storing labels

    #Each sample file is one row in data, and has one entry in labels
    print('Number of files total:', len(labels))
    all_labels = np.zeros(len(labels),dtype=int)
    for n, l in enumerate(label_set):
        label_idx_dic[n]=l
        all_labels[np.array([i for i, _ in enumerate(labels) if _ == l])] = n
    print('Labels and label indices', all_labels)
    
    return label_idx_dic, all_labels

In [12]:
label_dic, all_label = get_labels(labels,label_set)
# boy_label_dic, boy_all_label = get_labels(boy_labels,boy_label_set)

Number of files total: 9187
Labels and label indices [ 0  0  0 ... 31 31 31]


In [13]:
# save label dictionary
def save_dict_to_file(dic,dataset):
    with open('dict{}.txt'.format(dataset),'w') as file:
        file.write(json.dumps(dic))

In [15]:
# save dict
save_dict_to_file(label_dic,dataset)

#### Pad Zeros to Small Arrays

In [58]:
def data_x_y(data,all_labels):
    x_data = np.zeros((len(data),2808+1), dtype=float)

    for i,_d in enumerate(data):

        if (_d.shape[1]>2808):
            x_data[i,1:]=_d[0,0:2808]
        else :
            x_data[i,1:_d.shape[1]+1]=_d

        x_data[i,0]=all_labels[i]
        
    return x_data

#### SAVE DATASET

In [59]:
save_path=""

In [60]:
# save all features data
np.savetxt('data/{}/data41{}.csv'.format(save_path,'ADA'), data_x_y(data,all_label), delimiter=',',header=data_header)
# np.savetxt('data/{}/data41{}.csv'.format(save_path,'ADA_boy'), data_x_y(boy_data,boy_all_label), delimiter=',',header=data_header)

#### GENERATE  MIX NEW DATASET

In [65]:
features_df = pd.read_csv('data/data41.csv')
features_df_ada = pd.read_csv('data/data41ADA.csv')

In [66]:
x_data = np.vstack((features_df, features_df_ada))

In [67]:
print("normal dataset shape: {}".format(features_df.shape))
print("ada dataset shape: {}".format(features_df_ada.shape))
print("new mix dataset shape: {}".format(x_data.shape))

normal dataset shape: (21433, 2809)
ada dataset shape: (9187, 2809)
new mix dataset shape: (30620, 2809)


In [68]:
save_path=""
np.savetxt('data/{}/data41{}.csv'.format(save_path,'mix'), x_data, delimiter=',',header=data_header)

#### SPLIT TEST AND TRAIN DATA

In [69]:
save_path="mix"

In [70]:
x_data = pd.read_csv('data/data41mix.csv')
x_data.tail(5)

# target          0          1          2          3          4  \
30615      31.0  15.182958  19.488586  18.160939   5.327177 -20.082168   
30616      31.0  17.301883  21.527148  22.343778   2.823986 -14.873856   
30617      31.0  15.555322  13.875329  -0.415252  10.240820  -2.703395   
30618      31.0  12.355595   3.557199  16.481110  10.478649  -2.162406   
30619      31.0  14.699095  13.588537  16.883293   8.240038 -13.498826   

               5          6          7          8         9         10  \
30615  -6.150792  -5.297507   2.500258 -10.104961 -0.909694  -4.904961   
30616 -13.742811 -11.034787   2.880830  -9.300473 -8.324641 -17.562492   
30617   5.989147  -7.533577   8.411360  -8.067034  1.431822 -16.415926   
30618   9.217296   2.075993  11.069790   2.973149  2.737836 -13.632319   
30619  -5.192450 -18.113184  14.095021   0.087278  9.347218   0.039691   

             11         12         13         14         15         16  \
30615 -5.671047  -5.773044  16.910930  25.164618  14.349945   3.479753   
30616 -8.357018 -10.344853  18.505045  27.366775  14.897852   0.188628   
30617 -4.524558 -13.126099  16.462027  17.487794   4.264366   8.650438   
30618  8.454517 -10.150285  15.390828  13.292739  23.179047  14.591460   
30619  4.483913  -3.348220  16.348576  18.882994  14.935115  10.151788   

              17         18         19        20         21         22  \
30615 -19.295511 -16.609830  -8.295507  4.403316  -9.145877  -7.783766   
30616 -22.710197 -15.870356  -6.924357  8.497425 -12.567439 -18.308046   
30617  -5.975235  -3.778511 -10.310408  6.761522  -8.050992   0.889284   
30618   0.307310  -5.726399 -12.172158 -8.995741 -15.249847  -7.107584   
30619  -6.803818  -6.082939 -17.462365  3.787663 -11.233227  -8.836485   

              23        24         25         26         27         28  \
30615 -14.238339 -3.976433  -9.547944  18.114372  27.137889  14.556684   
30616 -21.523888  3.331131  -0.008734  19.430241  27.156074   3.842651   
30617 -18.084428 -9.206248 -17.160665  17.057136  20.896993   6.457598   
30618   2.824016  6.207443 -12.527986  16.583005  17.968015  22.337900   
30619 -13.748534 -7.848255  -7.525654  17.181665  21.992488  21.148702   

              29         30         31         32         33         34  \
30615   4.094584 -23.190555 -25.617183  -0.415620   5.868573  -9.187731   
30616   4.144387 -27.824611 -29.122912   3.753564   6.178923 -27.099383   
30617   9.195629  -9.671292  -8.496864 -13.626972   7.927847  -9.143352   
30618  14.736910  -3.857865  -7.713222 -12.444990 -11.115090 -10.954496   
30619  10.750578 -16.264812 -16.168276 -16.578294  13.787968  -7.296871   

              35         36         37         38         39         40  \
30615 -11.228821 -27.293348  -4.726736   4.341620  18.908116  29.042263   
30616 -14.011117 -14.674387  12.684359   6.835136  19.917722  25.360582   
30617  -0.011876 -20.250112  -9.704516 -18.255358  17.401593  23.604134   
30618  -3.139794  -6.337208  -4.060381  -6.502171  17.235186  21.380118   
30619 -14.602806 -15.147709  -6.522531 -16.978524  17.670341  22.690845   

              41         42         43         44         45         46  \
30615   4.674700   2.173251 -30.168553 -25.053780  10.249631  -0.609257   
30616  -5.143164  -0.328817 -23.733097 -22.815654   3.874323  -5.334044   
30617  19.120302   9.652107 -16.217481 -19.826127  -8.443366  16.077898   
30618  22.282808  13.753054  -6.429969  -8.474983  -9.871323  -8.201855   
30619  17.839079   3.866443 -16.385181 -14.110725 -13.356009   5.840110   

              47         48         49         50         51         52  \
30615 -26.167498 -12.933934 -19.981757   7.266830  10.315953  19.509186   
30616 -32.844039  -9.576097  -3.558956  17.443184   6.095139  20.363806   
30617  -8.584106  -9.828689 -17.405035  -5.436609 -20.718262  17.855439   
30618 -13.270154  -7.055302  -4.046097  -7.406262 -11.071084  17.471289   
30619 -13.336004 -14.213148 -19.104006  -6.515942 -10.

In [71]:
def create_test_train(x_data, save_path, data_header):
    from sklearn.model_selection import train_test_split

    X_train, X_test = train_test_split( x_data, test_size=0.15, random_state=42, shuffle=True)

    print('Size of training matrix:', X_train.shape)
    print('Size of testing matrix:', X_test.shape)

    # test data
    np.savetxt('data/{}/data41{}.csv'.format(save_path,save_path+'_train'), X_train, delimiter=',', header=data_header)
    np.savetxt('data/{}/data41{}.csv'.format(save_path,save_path+'_test'), X_test, delimiter=',', header=data_header)

In [72]:
create_test_train(x_data, save_path, data_header)

Size of training matrix: (26027, 2809)
Size of testing matrix: (4593, 2809)
